# Image classification using TensorFlow

### Springboard Career Track Capstone Project 2

#### Linear Model in TensorFlow

#### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
%matplotlib inline 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.core.pylabtools import figsize
figsize(12, 8)

#### Import data

Data acquired from https://www.cs.toronto.edu/~kriz/cifar.html

In [2]:
import tarfile
tar = tarfile.open("cifar-10-python.tar.gz")
tar.extractall()
tar.close()

#### Create function to unpickle data

In [3]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

#### Assign batch names, unpickle batches, and concatenate

In [4]:
f1 = 'cifar-10-batches-py\data_batch_1'
f2 = 'cifar-10-batches-py\data_batch_2'
f3 = 'cifar-10-batches-py\data_batch_3'
f4 = 'cifar-10-batches-py\data_batch_4'
f5 = 'cifar-10-batches-py\data_batch_5'
X_test = 'cifar-10-batches-py\test_batch'
label_names = 'cifar-10-batches-py\batches.meta'

In [5]:
b1 = unpickle(f1)
# b1

In [6]:
b1.keys()

dict_keys([b'batch_label', b'labels', b'data', b'filenames'])

In [7]:
train_y1 = b1.get(b'labels')

In [8]:
b1data = b1.get(b'data')

In [9]:
b2 = unpickle(f2)

In [10]:
train_y2 = b2.get(b'labels')

In [11]:
b2data = b2.get(b'data')

In [12]:
b3 = unpickle(f3)

In [13]:
train_y3 = b3.get(b'labels')

In [14]:
b3data = b3.get(b'data')

In [15]:
b4 = unpickle(f4)

In [16]:
train_y4 = b4.get(b'labels')

In [17]:
b4data = b4.get(b'data')

In [18]:
b5 = unpickle(f5)

In [19]:
train_y5 = b5.get(b'labels')

In [20]:
b5data = b5.get(b'data')

In [21]:
X_train = np.concatenate((b1data, b2data, b3data, b4data, b5data), axis=0)

In [22]:
type(X_train)

numpy.ndarray

In [23]:
X_train.shape

(50000, 3072)

In [24]:
X_train

array([[ 59,  43,  50, ..., 140,  84,  72],
       [154, 126, 105, ..., 139, 142, 144],
       [255, 253, 253, ...,  83,  83,  84],
       ..., 
       [ 35,  40,  42, ...,  77,  66,  50],
       [189, 186, 185, ..., 169, 171, 171],
       [229, 236, 234, ..., 173, 162, 161]], dtype=uint8)

In [25]:
y_train = np.concatenate((train_y1, train_y2, train_y3, train_y4, train_y5), axis=0)

In [26]:
type(y_train)

numpy.ndarray

In [27]:
y_train = y_train.reshape(50000,1)

In [28]:
y_train

array([[6],
       [9],
       [9],
       ..., 
       [9],
       [1],
       [1]])

In [29]:
X_test = unpickle(X_test)

OSError: [Errno 22] Invalid argument: 'cifar-10-batches-py\test_batch'

In [30]:
label_names = unpickle(label_names)

OSError: [Errno 22] Invalid argument: 'cifar-10-batches-py\x08atches.meta'

 #### Example from https://github.com/martin-gorner/tensorflow-mnist-tutorial/blob/master/mnist_1.0_softmax.py
 https://docs.google.com/presentation/d/1TVixw6ItiZ8igjp6U17tcgoFrLSaHWQmMOwjlgQY9co/pub?slide=id.g110257a6da_0_598

Attempt 1 - Not convolutional

In [31]:
learning_rate = .001
training_iteration = 30
batch_size = 100
display_step = 10

Initialize placeholder variables

In [32]:
X = tf.placeholder(tf.float32, [None, 3072])
W = tf.Variable(tf.zeros([3072,10]))
b = tf.Variable(tf.zeros([10]))
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


Placeholder for correct answers

In [33]:
Y_ = tf.placeholder(tf.float32, [None, 10])

Create model

In [34]:
Y = tf.nn.softmax(tf.matmul(tf.reshape(X,[-1,3072]), W) + b)

Loss function

In [35]:
cross_entropy = -tf.reduce_sum(Y_ * tf.log(Y))

% of correct answers found in batch

In [36]:
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(Y_,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

Add summary ops to collect data

In [37]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_step = optimizer.minimize(cross_entropy)

In [38]:
sess = tf.Session()
sess.run(init)

In [39]:
for i in range(10000):
    # load batch of images and correct answers
    batch_X, batch_Y = ??? # need to know what to fill in here
    train_data = {X: batch_X, Y_: batch_Y}
    
    # train
    sess.run(train_step, feed_dict=train_data)
    
    # success? add code to print it
    a,c = sess.run([accuracy, cross_entropy], feed=train_data)
    
    # success on test data?
    test_data={X: *test_images*, Y_: *test_labels*} # need to know what to fill in here
    a,c = sess.run([accuracy, cross_entropy], feed=test_data)

SyntaxError: invalid syntax (<ipython-input-39-dc7b049bf51e>, line 3)